## Model Building

In [ ]:
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
s3_bucket = ""
s3_prefix = "deepar_model"

role = ""

In [ ]:
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

In [ ]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image_uri = image_name,
    sagemaker_session = sagemaker_session,
    role = role,
    instance_count = 1,
    instance_type = "ml.c4.2xlarge",
    base_job_name = "deepar-trip-start1",
    output_path = s3_output_path,
)